In [1]:

from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.consts import END_KEY, PROMPT_FORMAT, RESPONSE_KEY_NL
from training.trainer import PROMPT_FORMAT, create_data_set_from_json_list

/opt/bo_ling/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = create_data_set_from_json_list("/opt/home/bo_ling/dolly/ma_data/so3_long.jsonl")
train_data

Exception: Please make sure that the input data file be in /home/bo_ling/dataset/michelangelo_so_long.jsonl,/home/bo_ling/dataset/docstrans.jsonl,/home/bo_ling/dataset/eats_prod.jsonl

In [48]:
local_output_dir="/opt/home/bo_ling/dolly_training/ma_helpdesk_2048/checkpoint-800"
model, tokenizer = load_model_tokenizer_for_generate(local_output_dir)

In [49]:
def generate_helpdesk_response(
    instruction: str,
    input_text: str,
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    do_sample: bool = True,
    max_new_tokens: int = 256,
    top_p: float = 0.92,
    top_k: int = 0,
    **kwargs,
) -> str:
    input_ids = tokenizer(PROMPT_FORMAT.format(instruction=instruction, input_text=input_text, output_text=""),
                          return_tensors="pt").input_ids.to("cuda")

    response_key_token_id = get_special_token_id(tokenizer, RESPONSE_KEY_NL)
    end_key_token_id = get_special_token_id(tokenizer, END_KEY)

    gen_tokens = model.generate(
        input_ids,
        pad_token_id=tokenizer.pad_token_id,
        # Ensure generation stops once it generates "### End"
        eos_token_id=end_key_token_id,
        do_sample=do_sample,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        top_k=top_k,
        **kwargs,
    )[0].cpu()

    # The response will be set to this variable if we can identify it.
    decoded = None

    # Find where "### Response:" is first found in the generated tokens.  Considering this is part of the prompt,
    # we should definitely find it.  We will return the tokens found after this token.
    response_pos = None
    response_positions = np.where(gen_tokens == response_key_token_id)[0]
    if len(response_positions) == 0:
        logger.warn(f"Could not find response key {response_key_token_id} in: {gen_tokens}")
    else:
        response_pos = response_positions[0]

    if response_pos:
        # Next find where "### End" is located.  The model has been trained to end its responses with this sequence
        # (or actually, the token ID it maps to, since it is a special token).  We may not find this token, as the
        # response could be truncated.  If we don't find it then just return everything to the end.  Note that
        # even though we set eos_token_id, we still see the this token at the end.
        end_pos = None
        end_positions = np.where(gen_tokens == end_key_token_id)[0]
        if len(end_positions) > 0:
            end_pos = end_positions[0]

        decoded = tokenizer.decode(gen_tokens[response_pos + 1 : end_pos]).strip()

    return decoded

In [50]:
count = 0
for d in train_data:
    instruction = d["instruction"]
    input_text= d["input"]
    generated = generate_helpdesk_response(instruction, input_text, model, tokenizer)
    expected = d['output']
    print("="*100)
    print("="*100)
    print("QUESTIONS:")
    print(input_text)
    print("\nGENERATED:")
    print(generated)
    print("\nEXPECTED:")
    print(expected)
    count += 1
    if count > 5:
        break

QUESTIONS:
Unable to build michelangelo core

GENERATED:
You need protobuf 2.5.0.   Documents workspace michelangelo michelangelo online-prediction   master  protoc --versionlibprotoc 2.5.0Also make sure you are connected to the VPN

EXPECTED:

You need protobuf 2.5.0.   Documents workspace michelangelo michelangelo online-prediction   master  protoc --versionlibprotoc 2.5.0Also make sure you are connected to the VPN
QUESTIONS:
Caffe training failed with message -- KeyError: Unable to open object  Object 'data' doesn't exist 

GENERATED:
You might hit this error when you train a Caffe deep learning model from michelangelo.Most likely your data is not image data and the draft model has a wrong configuration so that above code path  sampling image data  got executed which is wrong.To fix it, check  skip activation data  from the draft model dialog, then re-train your model.

EXPECTED:

You might hit this error when you train a Caffe deep learning model from michelangelo.Most likely your 

In [51]:
instruction = "Answer the following MA helpdesk questions:"
input_text = """Our ml-code devpod's drogon-cli is too old and running drogon command from it will produce error:
got Unexpected response from uSSO - {"version":"1.0.0","type":"server side error","message":"invalid response from Duo statuscode: 400","error":true}"""
output_text=""
print("QUESTIONS: \n")
print(input_text)
print("\nGENERATED:")
generate_helpdesk_response(instruction, input_text, model, tokenizer)

QUESTIONS: 

Our ml-code devpod's drogon-cli is too old and running drogon command from it will produce error:
got Unexpected response from uSSO - {"version":"1.0.0","type":"server side error","message":"invalid response from Duo statuscode: 400","error":true}

GENERATED:


"We have deprecated the JSON + HTTP endpoint from our API, and replaced them as THRIFT + HTTP endpoint.The thrift version of this endpoint is this onehttps:  engdocs.uberinternal.com michelangelo-rest thrift michelangelo_api.html#Fn_MichelangeloAPI_getGroupedDeploymentStatusesHere is the function wrapper of that endpoint in piper-core-pipelines repo.https:  code.uberinternal.com diffusion DAPIPE browse master michelangelo_platform lib rest_api.py$300We don't recommend people directly calling the"